In [1]:
# Dependencies
import pandas as pd
import numpy as np

from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float, Date
from sqlalchemy.orm import Session

import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Read in data 
measurements = pd.read_csv("Resources/clean_hawaii_measurements.csv")
stations = pd.read_csv("Resources/clean_hawaii_stations.csv")

In [3]:
measurements['id'] = measurements.index
measurements.head(3)

,station,date,prcp,tobs,id
0,USC00519397,2010-01-01,0.08,65,0
1,USC00519397,2010-01-02,0.00,63,1
2,USC00519397,2010-01-03,0.00,74,2


In [4]:
stations['id'] = stations.index
stations.head(3)

,station,name,latitude,longitude,elevation,id
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0,0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6,1
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0,2


In [5]:
# Create connection engine
engine = create_engine("sqlite:///./Resources/hawaii.sqlite")
conn = engine.connect()

In [6]:
# Set up dtype maps
dtype_map_measurements = {
    'id': Integer,
    'station': String,
    'date': Date, 
    'prcp': Float, 
    'tobs': Integer
}
dtype_map_stations = {
    'id': Integer,
    'station': String,
    'name': String, 
    'latitude': Float, 
    'longitude': Float, 
    'elevation': Float
}

In [7]:
# Create station and measurement tables within the database
# measurements.to_sql("measurements", conn, dtype=dtype_map_measurements, index=False,  if_exists='replace')
# stations.to_sql("stations", conn, dtype=dtype_map_stations, index=False,  if_exists='replace')

measurements.to_sql("measurements", engine, index=False,  if_exists='replace')
stations.to_sql("stations", engine, dtype=dtype_map_stations, index=False,  if_exists='replace')


In [8]:
inspector = inspect(engine)
inspector.get_table_names()

['measurements', 'stations']

In [9]:
# Get a list of column names and types
columns = inspector.get_columns('stations')
for c in columns:
    print(c['name'], c["type"])

print()
# Get a list of column names and types
columns = inspector.get_columns('measurements')
for c in columns:
    print(c['name'], c["type"])

station VARCHAR
name VARCHAR
latitude FLOAT
longitude FLOAT
elevation FLOAT
id INTEGER

station TEXT
date TEXT
prcp FLOAT
tobs BIGINT
id BIGINT


------------------

In [10]:
# Import and establish Base for which classes will be constructed 
# from sqlalchemy.ext.declarative import declarative_base
# Base = declarative_base()

from sqlalchemy.ext.automap import automap_base

# Reflect database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)

In [11]:
# from sqlalchemy.ext.declarative import declarative_base
# Base = declarative_base()

# Remap to add primary key
class Stations(Base):
    __tablename__ = "stations"
    __table_args__ = {"extend_existing": True}
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)   
    
    def __init__(self, station, name, latitude, longitude, elevation):
        self.station = station
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.elevation = elevation

class Measurements(Base):
    __tablename__ = "measurements"
    __table_args__ = {"extend_existing": True}
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __init__(self, station, date, prcp, tobs):
        self.station = station
        self.date = date
        self.prcp = pcrp
        self.tobs = tobs

Base.prepare()

# create tables
Base.metadata.create_all(engine)

In [12]:
# Create a session
session = Session(bind=engine)

In [13]:
m = session.query(Measurements).first()
m.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x10a9420b8>,
 'date': datetime.date(2010, 1, 1),
 'id': 0,
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [14]:
s = session.query(Stations).first()
s.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x10a942a58>,
 'elevation': 3.0,
 'id': 0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [15]:
precip = session.query(Measurements).filter(Measurements.date == '2010-01-09').all()
for p in precip:
#     print(p)
    print(p.date, p.prcp, p.id)

2010-01-09 0.0 6
2010-01-09 0.0 2693
2010-01-09 0.01 8007
2010-01-09 0.01 8351
2010-01-09 0.0 10923
2010-01-09 0.0 13695
2010-01-09 0.0 15627


In [16]:
s = session.query('stations')
s.__dict__

{'_entities': [<sqlalchemy.orm.query._ColumnEntity at 0x10a935400>],
 '_has_mapper_entities': False,
 '_mapper_adapter_map': {},
 '_polymorphic_adapters': {},
 '_primary_entity': None,
 'session': <sqlalchemy.orm.session.Session at 0x10a94db70>}

In [17]:
d = session.query(Stations)
for item in d:
    print(item.station, item.id)

USC00519397 0
USC00513117 1
USC00514830 2
USC00517948 3
USC00518838 4
USC00519523 5
USC00519281 6
USC00511918 7
USC00516128 8


In [18]:
Base.classes.keys()

[]